# Lyft: Understanding the data  and EDA

![](https://neurohive.io/wp-content/uploads/2019/07/Screenshot-from-2019-07-25-01-30-57.png)

### Credits:

**https://www.kaggle.com/t3nyks/lyft-working-with-map-api**<br>
**https://www.kaggle.com/jpbremer/lyft-scene-visualisations**<br>
**https://www.kaggle.com/pestipeti/pytorch-baseline-train**

This new Lyft competition is tasking us, the participants, to predict the motion of external cars, cyclists, pedestrians etc. to assist self-driving cars. This is a step ahead from last year's competition, where we were tasked with detecting three-dimensional objects, like stop signs, to teach AVs how to recognize these. 

**TIP: Use plt.imshow instead of IPython.display**

This is apparently the **largest collection of traffic agent motion data.** The files are stored in the .zarr file format with Python, which we can easily load using the Level 5 Kit (l5kit for the pip package). Within our training ZARRs, we have the agents, the masks for agents, frames and scenes (which you might recollect from last year) and traffic light faces.

The test ZARR however is almost practically the same format, but the only exclusion is that of the data masks. for the agents. 

# Get started with the data

Wait! Before we directly get into the fancy visualization and all it entails, why not watch a short YouTube video to enlighten us on the subject of operating an autonomous vehicle?

In [ ]:
from IPython.display import HTML
HTML('<center><iframe width="700" height="400" src="https://www.youtube.com/embed/tlThdr3O5Qo?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe></center>')

Seems like this car casually handles all the normal challenges a driver faces, and that too with remarkable accuracy. Over here, we are tasked with somethign to faciliate this sort of thing - **predicting the motion of extraneous vehicles and based on that, predicting the motion path of an AV.**  To predict the motion of these extraneous factors, there are many approaches which I shall discuss later, but for now let's dive in.

Here's a brief FAQ section about the dataset and all it entails:

**What is the structure of the dataset?**<br>
The dataset is structured as follows:
```
aerial_map
scenes
semantic_map
```

where each scene contains roughly a minute or so of information about the motion of several extraneous vehicles and the corresponding movement of the AV.

Under scenes, we have:
```
sample.zarr
test.zarr
train.zarr
validate.zarr
```

Now this ZARR format is a little bit interesting, as I am willing to fathom a guess most of the participants have never worked with these. Fear not, for they are very much interoperable with NumPy and the Lyft Level 5 Kit also gives us easy ways to handle the processing of the data. Of course, there also might be a few ways to use a Pandas DataFrame in the process, which leads up a road for LightGBM.

The train.zarr contains the agents, the mask for the agents, the frames, the scenes and the traffic light faces, which I'll go into more depth later.

<img src="https://self-driving.lyft.com/wp-content/uploads/2020/06/dataset-steps-longimg.png"></img>

Now, as we can see here, we have a sensor input (in the form of last year's sensory data) and they had to detect traffic agents in last year's competition. Here, we follow the next two steps of the process: predicting the agent motion and mapping out a path for the autonomous vehicle. The sensor is a LIDAR sensor, which basically gives us a rough perspective of the motion on the road. The sensor then feeds the data back to Lyft, who then collects the data from multiple sensors/cars all over Palo Alto, collates the data and gives it to us.

We may now import the lyft level 5 kit, and all that comes with it. We have quite a lot of imports and installations required...

**UPDATE: Finally got GPU to work by manually installing everything, adding utility scripts did not help at all. Took a painfully long time to get myself to realize that everything needs to be done in the kernel or things will break.**

In [ ]:
!pip install neptune-client segmentation_models_pytorch hydra-core kekas -U -q 
!pip install --target=/kaggle/working pymap3d==2.1.0 -q
!pip install --target=/kaggle/working strictyaml -q
!pip install --target=/kaggle/working protobuf==3.12.2 -q
!pip install --target=/kaggle/working transforms3d -q
!pip install --target=/kaggle/working zarr -q
!pip install --target=/kaggle/working ptable -q
!pip install --no-dependencies --target=/kaggle/working l5kit==1.1.0 --upgrade -q
!cp ../input/lyft-config-files/agent_motion_config.yaml config.yaml
import l5kit, os, albumentations as A
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from tqdm import tqdm
from l5kit.geometry import transform_points
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable
import matplotlib.pyplot as plt
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"
# get config
MONITORING = True # set this to false if you want to fork and train
if MONITORING:
    import utilsforlyft as U
cfg = load_config_data("../input/lyft-config-files/visualisation_config.yaml")
import plotly.offline as py
import omegaconf
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import torch
import torch.nn.functional as F
import torch.nn as nn
from catalyst import dl, data
from catalyst.utils import metrics
from torch.utils.data import DataLoader
from catalyst.dl import utils, BatchOverfitCallback
from torch.optim.lr_scheduler import OneCycleLR
import segmentation_models_pytorch as smp
from catalyst.core.callbacks.early_stop import EarlyStoppingCallback
if MONITORING:
    from catalyst.contrib.dl.callbacks.neptune_logger import NeptuneLogger
    from catalyst.contrib.dl.callbacks import WandbLogger
    neptune_logger = NeptuneLogger(
                    api_token=U.TOKEN + '=',  
                    project_name="trigram19/"+U.NAME_PROJ,
                    offline_mode=False, 
                    name=U.NAME,
                    params={'epoch_nr': 5}, 
                    properties={'data_source': 'lyft'},  
                    tags=['resnet']
                    )

from IPython.display import display, clear_output, HTML
import PIL
import matplotlib.pyplot as plt
from matplotlib import animation as ani, rc
import numpy as np
import warnings; warnings.filterwarnings('ignore')
from l5kit.rasterization.rasterizer_builder import _load_metadata
from kekas import Keker, DataOwner, DataKek
from kekas.utils import DotDict
from kekas.transformations import Transformer, to_torch, normalize

Here we go using some helpful functions to visualize the data.

In [ ]:
def plot_image(map_type, ax, agent=False):
    cfg["raster_params"]["map_type"] = map_type
    rast = build_rasterizer(cfg, dm)
    if agent:
        dataset = AgentDataset(cfg, zarr_dataset, rast)
    else:
        dataset = EgoDataset(cfg, zarr_dataset, rast)
    scene_idx = 2
    indexes = dataset.get_scene_indices(scene_idx)
    images = []
    for idx in indexes:    
        data = dataset[idx]
        im = data["image"].transpose(1, 2, 0)
        im = dataset.rasterizer.to_rgb(im)
        target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
        center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
        draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, yaws=data["target_yaws"])
        clear_output(wait=True)
        ax.imshow(im[::-1])
                
def animate_solution(images):
    def animate(i):
        im.set_data(images[i]) 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])    
    return ani.FuncAnimation(fig, animate, frames=len(images), interval=60)

def animation(type_):
    cfg["raster_params"]["map_type"] = type_
    rast = build_rasterizer(cfg, dm)
    dataset = EgoDataset(cfg, zarr_dataset, rast)
    scene_idx = 34
    indexes = dataset.get_scene_indices(scene_idx)
    images = []
    for idx in indexes:    
        data = dataset[idx]
        im = data["image"].transpose(1, 2, 0)
        im = dataset.rasterizer.to_rgb(im)
        target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
        center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
        draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, yaws=data["target_yaws"])
        clear_output(wait=True)
        images.append(PIL.Image.fromarray(im[::-1]))
    anim = animate_solution(images)
    return HTML(anim.to_jshtml())

def plot_with_tfms(tf):
    tfms = A.Compose(tf, keypoint_params=A.KeypointParams(format="xy", remove_invisible=False))
    im = tfms(image=train_dataset_a[0]['image'], keypoints=train_dataset_a[0]['target_positions'])
    keypoints = im['keypoints']
    im = train_dataset_a.rasterizer.to_rgb(im['image'].transpose(1, 2, 0))
    plt.imshow(im, cmap='Reds');
    


Now, let's get a sense of the configuration data. This will include metadata pertaining to the agents, the total time, the frames-per-scene, the scene time and the frame frequency.

In [ ]:
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"]);rasterizer = build_rasterizer(cfg, dm)
zarr_dataset = ChunkedDataset(dataset_path)
train_dataset_a = AgentDataset(cfg, zarr_dataset, rasterizer)
zarr_dataset.open()
print(zarr_dataset)

Now, however it's time for us to look at the scenes and analyze them in depth. Theoretically, we could create a nifty little data-loader to do some heavy lifting for us.

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(15, 10))
for i, key in enumerate(["py_semantic"]):
    plot_image(key, ax=axes)

So, there's a lot of information in this one image. I'll try my best to point everything out, but do notify me if I make any errors. OK, let's get started with dissecting the image:
+ We have an intersection of four roads over here.
+ The green blob represents the AV's motion, and we would require to predict the movement of the AV in these traffic conditions as a sample.

I don't exactly know what other inferences we can make without more detail on this data, so let's try a satellite-format viewing of these images. 

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(15, 10))
for i, key in enumerate(["py_satellite"]):
    plot_image(key, ax=axes)

Yes! This allows for far more detail than a simple plot without detail. I'd haphazard an educated guess, and make the following inferences:
+ Green still represents the autonomous vehicle (AV), and blue is primarily all the other cars/vehicles/exogenous factors we need to predict for.
+ My hypothesis is that the blue represents the path the vehicle needs to go through.
+ If we are able to accurately predict the path the vehicles go through, it will make it easier for an AV to compute its trajectory on the fly.

We also want to see how the whole charade of vehicles 

In [ ]:
animation("py_satellite")

So this is a demonstration of the movement of the other vehicles and (in relation to the movement and placement of the other vehicles) the movement of the AV. The AV is currently taking only a straight path in its motion, and a straight path seems logical with the movement and placement of other vehicles.

In [ ]:
animation("py_semantic")

We're also able to take a more low-level move by using the semantic option in the Lyft level 5 kit. Or... we can go even deeper and use the box rasterizer.

In [ ]:
animation("box_debug")

The semantic view is good for a less clustered view but if we want a more detailed, more high-level overview of the data we should perhaps try to use the satellite view voer semantic.

Now, how about from the agent perspective? This would be quite interesting to consider, as we're modeling from principally the agent perspective in most public notebooks so far.

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(15, 10))
for i, key in enumerate(["py_satellite"]):
    plot_image(key, ax=axes, agent=True)

So yes, I probably should save these as a GIF to visualize the agent movements. Let's try a simpler form of this and use the semantic view for the agent dataset.

Uh it seems the rasterizer renders rather well the satellite and semantic views, and both in conjunction help one to get a good sense of the positioning of each vehicle in relation to the road. You can easily understand the placement and motion of the vehicles and highway layout in satellite by taking a good look at the semantic view too.

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(15, 10))
for i, key in enumerate(["py_satellite", "py_semantic", 'stub_debug', "box_debug"]):
    plot_image(key, ax=axes, agent=True)

Again, box and stub will also give a good representaton of the data albeit with less low-level detail than the semantic view, seeing as the highways are not into much consideration here. The box view helps to just take a low-level look at the vehicles and their projected path whereas the stub view functions similarly to semantic. We can now proceed to taking a good look at the metadata provided by kkiller and potentially train a good model. The ones to check now will be stub and satellite to check.

Now as you can see with stub and box, under the hood the function uses the keys to create a rasterizer (for stub_debug for example StubRasterizer), which generates the AV surroundings and paths and passes it to an AgentDataset, with which we use to generate the predictions with our model. Also, there's a lot of meta-info about the rasterization that I want to have a look at, so let's look at metadata.

# Metadata Exploration

Now that we can explore the images, we can also get a little down and dirty when it comes to the ZARR files. It's rather simple to use with the Python library for exploring them, especially the fact that it's NumPy interoperable.

In [ ]:
print("scenes", zarr_dataset.scenes)
print("scenes[0]", zarr_dataset.scenes[0])

Also, a gentle note that we can use the ChunkedDataset to generate CSV files of scenes.

In [ ]:
import pandas as pd
scenes = zarr_dataset.scenes
scenes_df = pd.DataFrame(scenes)
scenes_df.columns = ["data"]; features = ['frame_index_interval', 'host', 'start_time', 'end_time']
for i, feature in enumerate(features):
    scenes_df[feature] = scenes_df['data'].apply(lambda x: x[i])
scenes_df.drop(columns=["data"],inplace=True)
print(f"scenes dataset: {scenes_df.shape}")
scenes_df.head()

But are we sure this is enough? Enough knowledge to satisfy us? No it's not. I will be using Kkiller's dataset for further tabular data exploration from henceforth.

In [ ]:
agents = pd.read_csv('../input/lyft-motion-prediction-autonomous-vehicles-as-csv/agents_0_10019001_10019001.csv')
agents

We have a literal wealth of information that we can use here to our benefits, including familiar features like:
1. x, y,  and z coords
2. yaw
3. probabilites of other extraneous factors.

In [ ]:
import seaborn as sns
colormap = plt.cm.magma
cont_feats = ["centroid_x", "centroid_y", "extent_x", "extent_y", "extent_z", "yaw"]
plt.figure(figsize=(16,12));
plt.title('Pearson correlation of features', y=1.05, size=15);
sns.heatmap(agents[cont_feats].corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True);

Here we can extrapolate that the variables **centroid_x** and **centroid_y** have strongly negative correlations, and the strongest correlations are between **extent_z** and **extent_x** more than any other, coming in at 0.4. We can also try using an XGBoost/LightGBM model as kkiller has demonstrated in his brilliant kernel as an alternative approach to the problem.

### centroid_x and centroid_y

In [ ]:
import seaborn as sns
plot = sns.jointplot(x=agents['centroid_x'][:1000], y=agents['centroid_y'][:1000], kind='hexbin', color='blueviolet')
plot.set_axis_labels('center_x', 'center_y', fontsize=16)

plt.show()

It seems like the two centroids have a somewhat strongly negative correlation and seemingly similar variable distributions. It seems that as such there is a negative correlation between both the variables.

### extent_x, extent_y and extent_z

In [ ]:
fig = plt.figure(figsize=(15, 15));
sns.distplot(agents['extent_x'], color='steelblue');
sns.distplot(agents['extent_y'], color='purple');

plt.title("Distributions of Extents X and Y");

It seems both the distributions of extent X and extent Y are heavily right skewed, as is centroid X. However, I have left out extent Z is order for readability of the plot, let's look at it now.

Try to smooth the data and get:

In [ ]:
fig = plt.figure(figsize=(15, 15));
sns.distplot(agents['extent_z'], color='steelblue');

plt.title("Distributions of Extents z");

Once again, we have a right-skewed distribution as is the same with all the `extent` variables. 

### yaw

In [ ]:
fig = plt.figure(figsize=(15, 15));
sns.distplot(agents['yaw'], color='steelblue');

plt.title("Distributions of Extents z");

So yes it seems like this distribution has several "protrusions" as I shall call them. We can now move on to exploring the frames data to check how feasible it is for our tabular purposes.

In [ ]:
frms = pd.read_csv("../input/lyft-motion-prediction-autonomous-vehicles-as-csv/frames_0_124167_124167.csv")
frms.head()

So here we have the ego rotations with regards to the centroids, which will be very interesting to consider. It seems like we will require to check multiple of these variables at once:

### ego_rotatations

First of all, we have nine ego rotation columns corresponding to each. So I would want to do a quick check of the correlation of these variables before moving on to some more high-level analyses.

In [ ]:
import seaborn as sns
colormap = plt.cm.magma
cont_feats = ["ego_rotation_xx", "ego_rotation_xy", "ego_rotation_xz", "ego_rotation_yx", "ego_rotation_yy", "ego_rotation_yz", "ego_rotation_zx", "ego_rotation_zy", "ego_rotation_zz"]
plt.figure(figsize=(16,12));
plt.title('Pearson correlation of features', y=1.05, size=15);
sns.heatmap(frms[cont_feats].corr(),linewidths=0.1,vmax=1.0, square=True, 
            cmap=colormap, linecolor='white', annot=True);


Things to note from this correlation analysis:
1. The rotation coordinates with `y` and `z` seem to be uncorrelated most of the time
2. The coordinates which have `x` are correlated strongly with the z-dimensional rotation (could this be indicative of something? I very much think so)

# Augmentation ideas

So the idea here is to augment the datas as discussed on the forum by @ryches (https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/discussion/188368) since albumentations can use the data with keypoint format.

Plot with the cutout transformation:

In [ ]:
plot_with_tfms([A.Cutout()])

Albumentations basically uses the keypoints provided in order to facilitate augmentation. (using cutout is dangerous because of loss of information possibilities btw, this is just for an example).

Let's see the pixel distributions for the above plot:

In [ ]:
tfms = A.Compose([A.Cutout()], keypoint_params=A.KeypointParams(format="xy", remove_invisible=False))
im = tfms(image=train_dataset_a[0]['image'], keypoints=train_dataset_a[0]['target_positions'])
keypoints = im['keypoints']
im = train_dataset_a.rasterizer.to_rgb(im['image'].transpose(1, 2, 0))
sns.distplot(im);

And how much they have changed from the original:

In [ ]:
sns.distplot(train_dataset_a[0]['image']);

Not much difference here except that we've shaved a lot off the `0.0` peak which is the effect of the cutout augmentatioin.

# Baseline model (source: [here](https://github.com/lyft/l5kit/blob/master/examples/agent_motion_prediction/agent_motion_prediction.ipynb))

![](https://raw.githubusercontent.com/catalyst-team/catalyst-pics/master/pics/catalyst_logo.png)

This is mainly me using the Lyft baseline model and training it, for the purpose of demonstrating how we can fit to a PyTorch model with the provided dataset format. Also, I am using the tool neptune.ai for monitoring the epoch progress.

Also using wonderful ML library catalyst and hydra, which helps your PyTorch training greatly. For now, let's get started on the modelling with some basic import setup:

In [ ]:
cfg2 = load_config_data("../input/lyft-config-files/agent_motion_config.yaml")
cfg2 = omegaconf.OmegaConf.create(cfg2)
train_cfg = omegaconf.OmegaConf.to_container(cfg2.train_data_loader)
validation_cfg = omegaconf.OmegaConf.to_container(cfg2.val_data_loader)
# Rasterizer
rasterizer = build_rasterizer(cfg2, dm)

Now we pretty-print the structure of our configuration. Hydra allows you to easily manage your configuration structure and schema whenever you train a machine learning model - it is also, as an additional benefit, part of the official **Pytorch Ecosystem.**

In [ ]:
print(cfg2.pretty())

This is basically initializing the rasterization process for the training data which basically functions to pass the .zarr files to our model and make it a coherent data format easily modular with our PyTorch setup.

In [ ]:
class LyftModel(torch.nn.Module):
    def __init__(self, cfg: omegaconf.dictconfig.DictConfig):
        super().__init__()
        self.backbone = smp.FPN(encoder_name="resnext50_32x4d", classes=1)
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels
        self.backbone.encoder.conv1 = nn.Conv2d(
            num_in_channels,
             self.backbone.encoder.conv1.out_channels,
            kernel_size= self.backbone.encoder.conv1.kernel_size,
            stride= self.backbone.encoder.conv1.stride,
            padding= self.backbone.encoder.conv1.padding,
            bias=False,
        ) 
        backbone_out_features = 14
        num_targets = 2 * cfg["model_params"]["future_num_frames"]
        self.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(in_features=14, out_features=4096),
        )
        self.backbone.segmentation_head = nn.Sequential(nn.Conv1d(56, 1, kernel_size=3, stride=2), nn.Dropout(0.2), nn.ReLU())
        self.logit = nn.Linear(4096, out_features=num_targets)
        self.logit_final = nn.Linear(128, 12)
        self.num_preds = num_targets * 3
    def forward(self, x):
        x = self.backbone.encoder.conv1(x)
        x = self.backbone.encoder.bn1(x)        
        x = self.backbone.encoder.relu(x)
        x = self.backbone.encoder.maxpool(x)        
        x = self.backbone.encoder.layer1(x)
        x = self.backbone.encoder.layer2(x)
        x = self.backbone.encoder.layer3(x)
        x = self.backbone.encoder.layer4(x)        
        x = self.backbone.decoder.p5(x)
        x = self.backbone.decoder.seg_blocks[0](x)
        x = self.backbone.decoder.merge(x)
        x = self.backbone.segmentation_head(x)
        x = self.backbone.encoder.maxpool(x)
        x = torch.flatten(x, 1)
        x = self.head(x)
        x = self.logit(x)   
        x = x.permute(1, 0)
        x = self.logit_final(x)
        return x

~~Unhide the above cell if you want to see the model, it's basically Peter's original work. All I am doing here is modifying the training pipeline to be Catalyst-compatible.~~

~~Update 17-09-2020: The model is roughly based on what kkiller has accomplished with PointNet and is basically me using a modified FPN network - to first segment (encode and decode) - and then use a simple MLP to classify. The model is roughly simple but it gives me pretty good results on loss, maybe something worth looking into?~~

The model is not eactly meant to segment + classify, that was my mistake in clarifying. It is simply a resnet50 + CBR block (Conv - BatchNorm - ReLU) + final part to get preds.

## Part 1: Catalyst training

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LyftModel(cfg2)
model.to(device)
train_zarr = ChunkedDataset(dm.require(train_cfg['key'])).open()
train_dataset = AgentDataset(cfg2, train_zarr, rasterizer)
del train_cfg['key']
subset = torch.utils.data.Subset(train_dataset, range(0, 1100))
train_dataloader = DataLoader(subset,
                              **train_cfg)
val_zarr = ChunkedDataset(dm.require(validation_cfg['key'])).open()
del validation_cfg['key']

val_dataset = AgentDataset(cfg2, val_zarr, rasterizer)
subset = torch.utils.data.Subset(val_dataset, range(0, 50))
val_dataloader = DataLoader(subset,
                              **validation_cfg)

This takes a very small subset of the data that we have here (mainly because this training pipeline is purely for demonstration purposes with regards to Catalyst).

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)

loaders = {
    "train": train_dataloader,
    "valid": val_dataloader
}

class LyftRunner(dl.SupervisedRunner):
    def predict_batch(self, batch):
        return self.model(batch[0].to(self.device).view(batch[0].size(0), -1))
    def _handle_batch(self, batch):
        x, y = batch['image'], batch['target_positions']
        y_hat = self.model(x).view(y.shape)
        target_availabilities = batch["target_availabilities"].unsqueeze(-1)
        criterion = torch.nn.MSELoss(reduction="none")
        loss = criterion(y_hat, y)
        loss = loss * target_availabilities
        loss = loss.mean()
        self.batch_metrics.update(
            {"loss": loss}
        )


Few things:
+ Configuration of the data loaders
+ Beginning the Catalyst process
+ Neptune, wandb  initialization

In [ ]:
%%time
device = utils.get_device()
runner = LyftRunner(device=device, input_key="image", input_target_key="target_positions", output_key="logits")
if MONITORING:

    runner.train(
        model=model,
        optimizer=optimizer,
        loaders=loaders,
        logdir="../working",
        num_epochs=4,
        verbose=True,
        load_best_on_end=True,
        callbacks=[neptune_logger, BatchOverfitCallback(train=10, valid=0.5), 
                  EarlyStoppingCallback(
            patience=2,
            metric="loss",
            minimize=True,
        ), WandbLogger(project="dertaismus",name= 'Example')
                  ]
    )
else:
    runner.train(
        model=model,
        optimizer=optimizer,
        loaders=loaders,
        logdir="../working",
        num_epochs=4,
        verbose=True,
        load_best_on_end=True,
        callbacks=[BatchOverfitCallback(train=10, valid=0.5), 
                  EarlyStoppingCallback(
            patience=2,
            metric="loss",
            minimize=True,
        )
                  ]
    )
    
# train for more steps and loss will not be low
# or at least not as pathetic as the loss over here    

And this is how the Neptune metrics look like:
![](https://i.imgur.com/uspP0q0.png)

To properly work with Neptune callbacks, replace the `U.` fields in the callback class with your API token, project name etc. Then, add it as a callback to Catalyst and watch the magic happen.

## Part 2: Kekas training

Kekas is a more simpler Fastai-esque way to train your model. Apart from having an extremely brilliant name, kekas gives you the simplicity of fastai and allows you to easily train and infer models.

In [ ]:
dataowner = DataOwner(train_dataloader, val_dataloader, None)
def step_fn(model: torch.nn.Module,
            batch: torch.Tensor) -> torch.Tensor:
    
    inp,t = batch["image"], batch["target_positions"]
    model.logit_final = nn.Linear(128, t.shape[0]).cuda()
    return model(inp).reshape(t.shape)

Keker is the equivalent of a Catalyst Runner.

In [ ]:
keker = Keker(model=model,
              dataowner=dataowner,
              criterion=torch.nn.MSELoss(),
              step_fn=step_fn,
              target_key="target_positions",
              opt=torch.optim.SGD,
              opt_params={"momentum": 0.99})

Similar to fastai - unfreeze model and then freeze to.

In [ ]:
keker.unfreeze(model_attr="backbone")

layer_num = -1
keker.freeze_to(layer_num, model_attr="backbone")

And now we `kek` (train) our model!

In [ ]:
keker.kek( lr=1e-3, 
            epochs=5,                  
            logdir='train_logs')

# loss computation is slightly faulty I'll try to fix it if possible
# issue is that we need to multiply loss by target availabilities like we did in Catalyst model
# however that does not work out of the box with kekas as far as I know, this is the main reason loss is going yoyo

keker.plot_kek('train_logs')